In [1]:
from imports import *

In [2]:
# Define names and filenames...

galaxy = 'ic5332'
galaxy_cat = galaxy
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name
cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/catalogue' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)
regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits' %root_dir
cluster_table_file = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_mask_file =  '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
catalog_mask_file = '%s/%s_mask.fits' %(dendro_dir, galaxy)

for prints in [hstha_file, muscat_file, musha_file, cutout_dir, 
               cutouts_hdus_dir, regions_file, regions_pickel_file, 
               sample_table_file, muscat_table_file, cluster_table_file, 
               cluster_mask_file, catalog_mask_file]:
    print(prints)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/hst_contsub/ic5332_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332-0.87asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts_hdus
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.reg
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/multiscale_stellar_ass

In [3]:
# Load regions, sample table and HDUs... 
hdus_cutouts  = cat_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions       = cat_misc.load_pickle(regions_pickel_file)
props_all     = QTable.read('%s/props_all.fits' %dendro_dir)
props_clusters = QTable.read(cluster_table_file)
hdu_cluster_mask = fits.open(cluster_mask_file)[0]
hdu_catalog_mask = fits.open(catalog_mask_file)[0]


# Get only regions in the props table...
region_IDs = props_all['region_ID']
for key in regions.keys():
    regions[key] = regions[key][region_IDs.astype(int).tolist()]
    

# Get cutouts... 
rerun_cutouts = True
if rerun_cutouts:

    hdus_cluster_mask = cat_cutouts.get_croppeddata_all(hdu_cluster_mask, regions) # Get cutouts of the cluster mask
    hdus_catalog_mask = cat_cutouts.get_croppeddata_all(hdu_catalog_mask, regions) # Get cutouts of the catalog mask

    hdus_cluster_mask_new = hdus_cluster_mask 
    hdus_catalog_mask_new = hdus_catalog_mask

    cat_misc.save_pickle(hdus_cluster_mask_new, '%s/hdus_cluster_mask_new.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_catalog_mask_new, '%s/hdus_catalog_mask_new.pickel' %cutout_dir)

else:

    hdus_cluster_mask_new = cat_misc.load_pickle('%s/hdus_cluster_mask_new.pickel' %cutout_dir)
    hdus_catalog_mask_new = cat_misc.load_pickle('%s/hdus_catalog_mask_new.pickel' %cutout_dir)

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_all.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel


Cropping regions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:15<00:00,  8.54it/s]


[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new.pickel
[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_catalog_mask_new.pickel


In [4]:
region_ID_all = np.unique(hdu_catalog_mask.data)
props_clusters_list = []

for i, region_ID in enumerate(props_all['region_ID']):

    # Get cluster IDs by masking the cluster mask with the region ID mask...
    cluster_IDs = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new)

    # If no cluster, fill with nans...
    if len(cluster_IDs) == 0: 
    
        props_clusters_new = cat_clusters.get_no_clusters(region_ID, props_clusters)

    # Only one cluster needs to be selected...
    if len(cluster_IDs) == 1: 

        props_clusters_new = cat_clusters.get_one_clusters(region_ID, props_clusters, cluster_IDs)
        
    # If more than one cluster, select the one with the minimum age...
    elif len(cluster_IDs) > 1:  

        props_clusters_new = cat_clusters.get_multi_clusters(region_ID, props_clusters, cluster_IDs)

    props_clusters_list += [props_clusters_new]

props_clusters_masked = vstack(props_clusters_list)
props_all_cluster = join(props_all, props_clusters_masked, keys='region_ID')

# Only take non-masked values in table
props_all_cluster_masked = props_all_cluster[~props_all_cluster['no_clusters']]
props_all_cluster_masked.write('%s/props_all_cluster.fits' %dendro_dir, overwrite=True)
props_all_cluster_masked

region_ID,x_max,y_max,x_com,y_com,ra_max,dec_max,ra_com,dec_com,npix,HA6562_FLUX_HST,HA6562_FLUX_ERR_HST,area_exact,radius_circ,flux_max,flux_min,flux_mean,radius_circ_pc,x_mom,y_mom,ra_mom,dec_mom,area_ellipse,major_sigma,minor_sigma,mean_sigma,position_angle,mean_sigma_pc,complexity_score,complexity_score_leaves,complexity_rms,complexity_std,flag_edge_hst,flag_touch_hst,gal_name,cen_x_MUSE,cen_y_MUSE,cen_ra_MUSE,cen_dec_MUSE,cen_x_geo_MUSE,cen_y_geo_MUSE,flag_edge_MUSE,flag_nearby_MUSE,flag_star_MUSE,deproj_dist_MUSE,deproj_phi_MUSE,region_area_MUSE,region_circ_rad_MUSE,HB4861_FLUX_MUSE,OIII5006_FLUX_MUSE,HA6562_FLUX_MUSE,NII6583_FLUX_MUSE,SII6716_FLUX_MUSE,SII6730_FLUX_MUSE,SIII9068_FLUX_MUSE,OI6300_FLUX_MUSE,NII5754_FLUX_MUSE,HEI5875_FLUX_MUSE,SIII6312_FLUX_MUSE,OI6363_FLUX_MUSE,OII7319_FLUX_MUSE,OII7330_FLUX_MUSE,HB4861_FLUX_ERR_MUSE,OIII5006_FLUX_ERR_MUSE,HA6562_FLUX_ERR_MUSE,NII6583_FLUX_ERR_MUSE,SII6716_FLUX_ERR_MUSE,SII6730_FLUX_ERR_MUSE,SIII9068_FLUX_ERR_MUSE,OI6300_FLUX_ERR_MUSE,NII5754_FLUX_ERR_MUSE,HEI5875_FLUX_ERR_MUSE,SIII6312_FLUX_ERR_MUSE,OI6363_FLUX_ERR_MUSE,OII7319_FLUX_ERR_MUSE,OII7330_FLUX_ERR_MUSE,HA6562_SIGMA_MUSE,NII6583_SIGMA_MUSE,OIII5006_SIGMA_MUSE,HA6562_VEL_MUSE,NII6583_VEL_MUSE,OIII5006_VEL_MUSE,HA6562_SIGMA_ERR_MUSE,NII6583_SIGMA_ERR_MUSE,OIII5006_SIGMA_ERR_MUSE,HA6562_VEL_ERR_MUSE,NII6583_VEL_ERR_MUSE,OIII5006_VEL_ERR_MUSE,HB4861_FLUX_CORR_MUSE,OIII5006_FLUX_CORR_MUSE,HA6562_FLUX_CORR_MUSE,NII6583_FLUX_CORR_MUSE,SII6716_FLUX_CORR_MUSE,SII6730_FLUX_CORR_MUSE,SIII9068_FLUX_CORR_MUSE,OI6300_FLUX_CORR_MUSE,NII5754_FLUX_CORR_MUSE,HEI5875_FLUX_CORR_MUSE,SIII6312_FLUX_CORR_MUSE,OI6363_FLUX_CORR_MUSE,OII7319_FLUX_CORR_MUSE,OII7330_FLUX_CORR_MUSE,HB4861_FLUX_CORR_ERR_MUSE,OIII5006_FLUX_CORR_ERR_MUSE,HA6562_FLUX_CORR_ERR_MUSE,NII6583_FLUX_CORR_ERR_MUSE,SII6716_FLUX_CORR_ERR_MUSE,SII6730_FLUX_CORR_ERR_MUSE,SIII9068_FLUX_CORR_ERR_MUSE,OI6300_FLUX_CORR_ERR_MUSE,NII5754_FLUX_CORR_ERR_MUSE,HEI5875_FLUX_CORR_ERR_MUSE,SIII6312_FLUX_CORR_ERR_MUSE,OI6363_FLUX_CORR_ERR_MUSE,OII7319_FLUX_CORR_ERR_MUSE,OII7330_FLUX_CORR_ERR_MUSE,EBV_MUSE,EBV_ERR_MUSE,met_scal_MUSE,met_scal_err_MUSE,logq_D91_MUSE,logq_D91_err_MUSE,t_N2_MUSE,t_S3_MUSE,t_N2_err_MUSE,t_S3_err_MUSE,BPT_NII_MUSE,BPT_SII_MUSE,BPT_OI_MUSE,LOG_SIGMA_SFR_CHAB_MUSE,LOG_SIGMA_STMASS_CHAB_MUSE,5thNN_dist_MUSE,NN_number_MUSE,Environment_MUSE,r_R25_MUSE,r_reff_MUSE,HA6562_LUMINOSITY_MUSE,Delta_met_scal_MUSE,pc_per_pixel_MUSE,EW_HA6562_raw_MUSE,EW_HA6562_raw_ERR_MUSE,EW_HB4861_raw_MUSE,EW_HB4861_raw_ERR_MUSE,EW_HA6562_fit_MUSE,EW_HA6562_fit_ERR_MUSE,EW_HB4861_fit_MUSE,EW_HB4861_fit_ERR_MUSE,EW_HA6562_bgcorr_MUSE,EW_HA6562_bgcorr_ERR_MUSE,EW_HB4861_bgcorr_MUSE,EW_HB4861_bgcorr_ERR_MUSE,HII_class_v2_MUSE,HII_class_v3_MUSE,HA6562_FLUX_CORR_HST,HA6562_FLUX_ERR_CORR_HST,HA6562_LUMINOSITY_HST,HA6562_LUMINOSITY_ERR_HST,region_circ_rad_pc_MUSE,reg_id,reg_x,reg_y,reg_ra,reg_dec,reg_area,reg_rad,NUV_dolflux_mjy,NUV_dolflux_mjy_err,U_dolflux_mjy,U_dolflux_mjy_err,B_dolflux_mjy,B_dolflux_mjy_err,V_dolflux_mjy,V_dolflux_mjy_err,I_dolflux_mjy,I_dolflux_mjy_err,NUV_dolmag_vega,NUV_dolmag_vega_err,U_dolmag_vega,U_dolmag_vega_err,B_dolmag_vega,B_dolmag_vega_err,V_dolmag_vega,V_dolmag_vega_err,I_dolmag_vega,I_dolmag_vega_err,reg_dolflux_Age_MinChiSq,reg_dolflux_Age_MinChiSq_err,reg_dolflux_Mass_MinChiSq,reg_dolflux_Mass_MinChiSq_err,reg_dolflux_Ebv_MinChiSq,reg_dolflux_Ebv_MinChiSq_err,reg_dolflux_ChiSq_Reduced,reg_dolflux_Age_MinChiSq_ave,reg_dolflux_Mass_MinChiSq_sum,no_clusters,one_clusters,multiple_clusters,reg_dolflux_Age_MinChiSq_massweighted
,pix,pix,pix,pix,deg,deg,deg,deg,pix,erg / (s cm2),erg / (s cm2),arcsec2,arcsec,erg / (s cm2),erg / (s cm2),erg / (s cm2),pc,pix,pix,deg,deg,arcsec2,arcsec,arcsec,arcsec,deg,pc,,,,,,,,pix,pix,deg,deg,pix,pix,,,,arcsec,rad,pix2,arcsec,1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg / (s cm2),1e-20 erg